Keras (ja TensorFlow) jatkoa
======================
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1229px-TensorFlowLogo.svg.png" 
alt="TensorFlow" width="400"/>
![Keras](https://upload.wikimedia.org/wikipedia/commons/c/c9/Keras_Logo.jpg "Keras")

Asentaaksesi tarvittavat paketit omalla koneellasi harjoituksen suorittamista varten:
```
$ pip3 install scikit-learn tensorflow==2.0.0rc1 numpy matplotlib==2.2.2
```

## Tekstin luokittelua
Tämä harjoitus on muokattu käyttäen [tutoriaalia](https://www.tensorflow.org/tutorials/keras/basic_text_classification)

Tällä kertaa aineistona käytetään [imdb](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)-aineistoa, joka sisältää sanallisia arvosteluja numeroiksi muutettuna ja luokkana on joko positiivinen tai negatiivinen arvostelu.



In [30]:
from tensorflow import keras
imdb = keras.datasets.imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

In [31]:
# datan katselua
print("Aineistojen koko: opetus:{}, testi:{}".format(len(y_train), len(y_test)))
print("Ensimmäinen havainto", X_train[0])
print("Ensimmäisten pituuksia", [len(x) for x in X_train[:5]])

Aineistojen koko: opetus:25000, testi:25000
Ensimmäinen havainto [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178,

In [32]:
# Sanasto
word_index = imdb.get_word_index()

In [33]:
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3

reverse_index = {v: k for (k, v) in word_index.items()}

def translate(sample):
    return ' '.join([reverse_index.get(i, '<not found>') for i in sample])

In [34]:
print("Ensimmäinen havainto sanoina:\n", translate(X_train[0]))

Ensimmäinen havainto sanoina:
 <START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNKNOWN> is an amazing actor and now the same being director <UNKNOWN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNKNOWN> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNKNOWN> to the two little boy's that played the <UNKNOWN> of norman and paul they were just brilliant children are often left out of the <UNKNOWN> list i think because the stars that play them all grown up are such a big profile for the whole film but t

In [35]:
# maksimipituus
print(max([len(x) for x in X_train]))

2494


In [36]:
# Pituudet eri, joten [](https://keras.io/preprocessing/sequence/#pad_sequences)
# Myös määrittää maksimipituuden
X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                            value=word_index["<PAD>"],
                                            padding='post',
                                            maxlen=256)

In [37]:
X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                            value=word_index["<PAD>"],
                                            padding='post',
                                            maxlen=256)

In [38]:
print(X_train[0])
print(translate(X_train[0]))

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [44]:
def create_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Embedding(10000, 16))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(16, activation="relu"))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    
    print(model.summary())
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
    return model

In [46]:
model = create_model()
history = model.fit(X_train,
                    y_train,
                    epochs=2,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
None
Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 5s 244us/sample - loss: 0.6913 - acc: 0.5758 - val_loss: 0.6872 - val_acc: 0.6500
Epoch 2/2
20000/20000 [==============================

In [48]:
# evaluate
evaluation = model.evaluate(X_test[:1000], y_test[:1000], batch_size=100)
print(evaluation)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.6712392270565033, 0.74]


## Regressio
Ehkä [tämä](https://www.tensorflow.org/tutorials/keras/basic_regression) tutoriaali?